In [2]:
import pandas as pd
import os
import assets.helper as b3


In [5]:
file_path = 'datasets/b3_companies.zip'

In [9]:
df = pd.read_pickle(file_path)  # Try to read the file as a pickle.


In [13]:
b3_companies = df

In [14]:
b3_companies.fillna('', inplace=True)


In [18]:
b3_companies.reset_index(drop=True, inplace=True)


In [21]:
b3_companies.drop_duplicates(inplace=True)


In [22]:
b3_companies

,pregao,company_name,cvm,listagem,ticker,tickers,asin,cnpj,site,setor,subsetor,segmento,atividade,escriturador,url
0,2W ENERGIA,2W ENERGIA SA,917581.0,,2WAV,,,,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
1,3R PETROLEUM,3R PETROLEUM OLEO E GAS SA,25291.0,CIA NOVO MERCADO,RRRP,RRRP3,BRRRRPACNOR5,12.091.809/0001-55,,Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,Exploração e produção de petróleo e gás natural.,ITAU CORRETORA ACOES,https://sistemaswebb3-listados.b3.com.br/liste...
2,3ZREALTY,3Z REALTY DESENVOLVIMENTO IMOBILIARIO SA,908830.0,,TZRD,,,08.145.855/0001-12,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
3,524 PARTICIP,524 PARTICIPACOES SA,16284.0,CIA BALCAO ORG TRADICIONAL,QVQP,QVQP3B,BRQVQPACNOR1,01.851.771/0001-55,http://www.524participacoes.com.br/home/index....,Outros,Outros,Outros,Holding - participação em outras sociedades,ITAU CORRETORA ACOES,https://sistemaswebb3-listados.b3.com.br/liste...
4,GERADORA,A GERADORA ALUGUEL DE MAQUINAS SA,904110.0,,GRDR,,,33.845.322/0001-90,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,WIZ CO,WIZ CO PARTICIPACOES E CORRETAGEM DE SEGUROS SA,23590,CIA NOVO MERCADO,WIZC,WIZC3,BRWIZCACNOR5,42.278.473/0001-03,www.wizsolucoes.com.br,Financeiro,Previdência e Seguros,Corretoras de Seguros e Resseguros,Corretagem de seguros de todos os ramos. asses...,ITAU CORRETORA ACOES,https://sistemaswebb3-listados.b3.com.br/liste...
2386,2W ENERGIA,2W ENERGIA SA,917581,,2WAV,,,,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
2387,AMER TOWER,AMERICAN TOWER DO BRASIL CESSAO DE INFRAESTRUTUR,917658,,ATBR,,,,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
2388,CONSTR BRA,CONSTRUTORA CENTRAL DO BRASIL SA,917650,,CBRL,,,,,,,,,,https://sistemaswebb3-listados.b3.com.br/liste...
